In [1]:
import gym
import pandas
import numpy as np

env = gym.make('CartPole-v0')

In [2]:
num_episodes = 10000

life_memory = []

for i in range(num_episodes):
    old_observation = env.reset()
    done = False
    tot_reward = 0
    ep_memory = []
    while not done:
        new_action = env.action_space.sample()
        observation, reward, done, info = env.step(new_action)
        tot_reward += reward
        
        ep_memory.append({
            "obs0": old_observation[0],
            "obs1": old_observation[1],
            "obs2": old_observation[2],
            "obs3": old_observation[3],
            "action": new_action,
            "reward": reward,
            "episode": i,
        })
        old_observation = observation
        
    for ep_mem in ep_memory:
        ep_mem["tot_reward"] = tot_reward
        
    life_memory.extend(ep_memory)
    
memory_df = pandas.DataFrame(life_memory)


In [3]:
memory_df.groupby("episode").reward.sum().mean()
memory_df["comb_reward"] = .5*memory_df.reward + 1.2 * memory_df.tot_reward

In [4]:
x_mat = memory_df[["obs0", "obs1", "obs2", "obs3", "action"]].to_numpy()
y = memory_df['comb_reward'].to_numpy()

### Two hiden layers Neural Network

In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers

# optimizer = tf.keras.optimizers.RMSprop(0.001)

def build_model2():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape=(x_mat.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='relu'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [6]:
model2 = build_model2()
model2.fit(x_mat, y, epochs = 100)

Epoch 1/100
6944/6944 [==============================] - 6s 869us/step - loss: 280.4310 - mae: 12.4106
Epoch 2/100
6944/6944 [==============================] - 6s 866us/step - loss: 218.5285 - mae: 10.81840s - loss: 218.7545 - 
Epoch 3/100
6944/6944 [==============================] - 7s 947us/step - loss: 208.3697 - mae: 10.5419
Epoch 4/100
6944/6944 [==============================] - 6s 916us/step - loss: 203.1162 - mae: 10.4027
Epoch 5/100
6944/6944 [==============================] - 6s 852us/step - loss: 201.3332 - mae: 10.3487
Epoch 6/100
6944/6944 [==============================] - 6s 834us/step - loss: 200.3968 - mae: 10.3203
Epoch 7/100
6944/6944 [==============================] - 6s 857us/step - loss: 199.3587 - mae: 10.2989
Epoch 8/100
6944/6944 [==============================] - 6s 844us/step - loss: 198.3084 - mae: 10.2708
Epoch 9/100
6944/6944 [==============================] - 6s 835us/step - loss: 197.6679 - mae: 10.2532
Epoch 10/100
6944/6944 [===========================

6944/6944 [==============================] - 6s 887us/step - loss: 193.3936 - mae: 10.1246
Epoch 80/100
6944/6944 [==============================] - 6s 845us/step - loss: 193.3840 - mae: 10.1285
Epoch 81/100
6944/6944 [==============================] - 6s 901us/step - loss: 193.2666 - mae: 10.1236
Epoch 82/100
6944/6944 [==============================] - 6s 871us/step - loss: 193.2042 - mae: 10.1169
Epoch 83/100
6944/6944 [==============================] - 6s 857us/step - loss: 193.1539 - mae: 10.1191
Epoch 84/100
6944/6944 [==============================] - 6s 870us/step - loss: 193.1528 - mae: 10.1210
Epoch 85/100
6944/6944 [==============================] - 6s 902us/step - loss: 193.1635 - mae: 10.1170
Epoch 86/100
6944/6944 [==============================] - 6s 838us/step - loss: 193.1681 - mae: 10.1150
Epoch 87/100
6944/6944 [==============================] - 6s 882us/step - loss: 193.0849 - mae: 10.1163
Epoch 88/100
6944/6944 [==============================] - 6s 883us/step - los

In [8]:
import datetime
n = datetime.datetime.now

In [9]:
t = n()
num_episodes = 100
n_life_memory2 = []
n_env = gym.make('CartPole-v0')
for i in range(num_episodes):
    n_old_observation = n_env.reset()
    n_done = False
    n_tot_reward = 0
    n_em_memory2 = []
    while not n_done:
        pred_in = [list(n_old_observation) + [i] for i in range(2)]
        n_new_action = np.argmax([model2.predict(pred_in)])
        n_observation, n_reward, n_done, n_info = n_env.step(n_new_action)
        n_tot_reward += n_reward
        n_em_memory2.append({
            "obs0": n_old_observation[0],
            "obs1": n_old_observation[1],
            "obs2": n_old_observation[2],
            "obs3": n_old_observation[3],
            "action": n_new_action,
            "reward": n_reward,
            "episode": i,
        })
        
        n_old_observation = n_observation

    for n_em in n_em_memory2:
        n_em["tot_reward"] = n_tot_reward
    n_life_memory2.extend(n_em_memory2)
n_memory_df2 = pandas.DataFrame(n_life_memory2)
print(f"Training time: {n() - t}s")

Training time: 0:10:12.948610s


In [10]:
n_memory_df2.groupby('episode').reward.sum().mean()

166.88

In [11]:
(n_memory_df2.groupby('episode').reward.sum() >= 200).value_counts()

False    61
True     39
Name: reward, dtype: int64

### Three hiden layers Neural Network

In [9]:
import tensorflow as tf
from tensorflow.keras import models, layers

def build_model3():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape=(x_mat.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [10]:
model3 = build_model3()
model3.fit(x_mat, y, epochs = 100)

Epoch 1/100
710/710 [==============================] - 1s 895us/step - loss: 408.7547 - mae: 14.7005
Epoch 2/100
710/710 [==============================] - 1s 1ms/step - loss: 233.1066 - mae: 11.1831
Epoch 3/100
710/710 [==============================] - 1s 922us/step - loss: 225.9711 - mae: 10.9368
Epoch 4/100
710/710 [==============================] - 1s 900us/step - loss: 222.1545 - mae: 10.8156
Epoch 5/100
710/710 [==============================] - 1s 897us/step - loss: 219.3632 - mae: 10.7486
Epoch 6/100
710/710 [==============================] - 1s 948us/step - loss: 216.1697 - mae: 10.6611
Epoch 7/100
710/710 [==============================] - 1s 916us/step - loss: 214.0153 - mae: 10.6128
Epoch 8/100
710/710 [==============================] - 1s 942us/step - loss: 211.7704 - mae: 10.5367
Epoch 9/100
710/710 [==============================] - 1s 1ms/step - loss: 209.7602 - mae: 10.4793
Epoch 10/100
710/710 [==============================] - 1s 905us/step - loss: 208.2997 - mae: 1

In [ ]:
import datetime
n = datetime.datetime.now

In [18]:
t = n()
num_episodes = 100
n_life_memory3 = []
n_env = gym.make('CartPole-v0')
for i in range(num_episodes):
    n_old_observation = n_env.reset()
    n_done = False
    n_tot_reward = 0
    n_em_memory3 = []
    while not n_done:
        pred_in = [list(n_old_observation) + [i] for i in range(2)]
        n_new_action = np.argmax([model3.predict(pred_in)])
        n_observation, n_reward, n_done, n_info = n_env.step(n_new_action)
        n_tot_reward += n_reward
        n_em_memory3.append({
            "obs0": n_old_observation[0],
            "obs1": n_old_observation[1],
            "obs2": n_old_observation[2],
            "obs3": n_old_observation[3],
            "action": n_new_action,
            "reward": n_reward,
            "episode": i,
        })
        
        n_old_observation = n_observation

    for n_em in n_em_memory3:
        n_em["tot_reward"] = n_tot_reward
    n_life_memory3.extend(n_em_memory3)
n_memory_df3 = pandas.DataFrame(n_life_memory3)
print(f"Training time: {n() - t}s")

Training time: 0:09:58.996290s


In [19]:
n_memory_df3.groupby('episode').reward.sum().mean()

154.55

In [20]:
(n_memory_df3.groupby('episode').reward.sum() >= 200).value_counts()

False    62
True     38
Name: reward, dtype: int64

### Neural Network from scratch
https://www.youtube.com/watch?v=Wo5dMEP_BbI&list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3&index=1

In [26]:
x_mat = memory_df[["obs0", "obs1", "obs2", "obs3", "action"]].to_numpy()
y = memory_df['comb_reward'].to_numpy()

In [35]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# def loss_fn(y_true, y_pred, eps=1e-16):
#     """
#     Loss function we would like to optimize (minimize)
#     We are using Logarithmic Loss
#     http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss
#     """
#     y_pred = np.maximum(y_pred,eps)
#     y_pred = np.minimum(y_pred,(1-eps))
#     return -(np.sum(y_true * np.log(y_pred)) + np.sum((1-y_true)*np.log(1-y_pred)))/len(y_true)

def relu(x):
    return np.maximum(0, x)

def deri_relu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def forward_pass(W_1, W_2):
    global x_mat
    global y
    global num_

    z_2 = np.dot(x_mat, W_1)   # (input_size , 5) x (5, 64) = (input_size , 64)
    a_2 = sigmoid(z_2)          # (input_size , 64)
    z_3 = np.dot(a_2, W_2)      # (input_size , 64) x (64, ) = (221input_size191 , )
    y_pred = sigmoid(z_3).reshape((len(x_mat),))      # (input_size , )
    J_z_3_grad = -y + y_pred      # (input_size , )
    J_W_2_grad = np.dot(J_z_3_grad, a_2)      # (input_size , ) x (input_size , 64) = (64, )
    a_2_z_2_grad = sigmoid(z_2)*(1 - sigmoid(z_2))  # (input_size , 64) * (input_size , 64) = (input_size , 64)
                                                    #     J_W_1_grad = a_1 * a_2_z_2_grad * W_2 * J_z_3_grad
                                                    #     J_W_1_grad = np.dot(x_mat, a_2_z_2_grad).dot(W_2).dot(J_z_3_grad)
    J_W_1_grad = (np.dot((J_z_3_grad).reshape(-1,1), W_2.reshape(-1,1).T)*a_2_z_2_grad).T.dot(x_mat).T
                        # (input_size , )->(input_size , 1) x (64 x 1).T
                                                   # = (input_size , 64) * (input_size , 64)
                                                                  # = (input_size , 64).T = (64, input_size ) x (input_size , 5)
                                                                  # = (64, 5).T = (5, 64)
    gradient = (J_W_1_grad, J_W_2_grad)
    
    return y_pred, gradient

In [ ]:
# How to update bias??

In [110]:
W_1 = np.random.uniform(-1,1,size=(5,64))
W_2 = np.random.uniform(-1,1,size=(64))
bias = 0.3

In [119]:
import datetime
n = datetime.datetime.now
t = n()

np.random.seed(1241)

num_iter = 50
learning_rate = .001
# x_mat = x_mat_full

total_pred = []
for i in range(num_iter):
    y_pred, (J_W_1_grad, J_W_2_grad) = forward_pass(W_1, W_2)
    
    W_1 = W_1 - learning_rate * J_W_1_grad
    W_2 = W_2 - learning_rate * J_W_2_grad
    
    total_pred.append(y_pred)
print(f"Training time: {n() - t}s")

C:\Users\moon\AppData\Local\Temp/ipykernel_28008/916779298.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Training time: 0:00:10.780250s


In [120]:
def nn_predict(pred_in):
    z_2 = np.dot(pred_in[0], W_1)
    a_2 = sigmoid(z_2)
    z_3 = np.dot(a_2, W_2)
    y_pred1 = sigmoid(z_3)
    
    z2_2 = np.dot(pred_in[1], W_1)
    a2_2 = sigmoid(z2_2)
    z2_3 = np.dot(a2_2, W_2)
    y_pred2 = sigmoid(z2_3)
    
    return y_pred1, y_pred2

In [121]:
t = n()
num_episodes = 1000
n_life_memory = []
n_env = gym.make('CartPole-v0')
for i in range(num_episodes):
    n_old_observation = n_env.reset()
    n_done = False
    n_tot_reward = 0
    n_em_memory = []
    while not n_done:
        pred_in = [list(n_old_observation) + [i] for i in range(2)]
        n_new_action = np.argmax([nn_predict(pred_in)])
        n_observation, n_reward, n_done, n_info = n_env.step(n_new_action)
        n_tot_reward += n_reward
        n_em_memory.append({
            "obs0": n_old_observation[0],
            "obs1": n_old_observation[1],
            "obs2": n_old_observation[2],
            "obs3": n_old_observation[3],
            "action": n_new_action,
            "reward": n_reward,
            "episode": i,
        })
        
        n_old_observation = n_observation

    for n_em in n_em_memory:
        n_em["tot_reward"] = n_tot_reward
    n_life_memory.extend(n_em_memory)
n_memory_df = pandas.DataFrame(n_life_memory)
print(f"Training time: {n() - t}s")

C:\Users\moon\AppData\Local\Temp/ipykernel_28008/916779298.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Training time: 0:00:01.298666s


In [122]:
n_memory_df.groupby('episode').reward.sum().mean()

21.07

In [123]:
(n_memory_df.groupby('episode').reward.sum() >= 200).value_counts()

False    998
True       2
Name: reward, dtype: int64

In [116]:
t = n()
num_episodes = 1000
n_life_memory = []
n_env = gym.make('CartPole-v0')
for i in range(num_episodes):
    n_old_observation = n_env.reset()
    n_done = False
    n_tot_reward = 0
    n_em_memory = []
    while not n_done:
        n_new_action = env.action_space.sample()
        n_observation, n_reward, n_done, n_info = n_env.step(n_new_action)
        n_tot_reward += n_reward
        n_em_memory.append({
            "obs0": n_old_observation[0],
            "obs1": n_old_observation[1],
            "obs2": n_old_observation[2],
            "obs3": n_old_observation[3],
            "action": n_new_action,
            "reward": n_reward,
            "episode": i,
        })
        
        n_old_observation = n_observation

    for n_em in n_em_memory:
        n_em["tot_reward"] = n_tot_reward
    n_life_memory.extend(n_em_memory)
ran_memory_df = pandas.DataFrame(n_life_memory)
print(f"Training time: {n() - t}s")

Training time: 0:00:00.365990s


In [117]:
ran_memory_df.groupby('episode').reward.sum().mean()

21.907

In [118]:
(ran_memory_df.groupby('episode').reward.sum() >= 200).value_counts()

False    1000
Name: reward, dtype: int64